In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-01-31 21:26:23--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   151MB/s    in 0.2s    

2023-01-31 21:26:24 (151 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [12]:
# add your code here - consider creating a new cell for each section of code
df_books.dropna(inplace=True)
df_ratings.dropna(inplace=True)

In [13]:
print(f"Total number of ratings : {len(df_ratings)}")
raters = df_ratings['user'].value_counts()
print(f"Total number of raters : {len(raters)}")
raters.sort_values(ascending=False).head()
print(f"Number of raters that will be dropped : {len(raters[raters < 200])}")
print(f"The numbers of ratings that will be dropped : {df_ratings['user'].isin(raters[raters < 200].index).sum()}")
df_ratings_rm = df_ratings[~df_ratings['user'].isin(raters[raters < 200].index)]
print(f"The numbers of remaining ratings : {len(df_ratings_rm)}")

Total number of ratings : 1149780
Total number of raters : 105283
Number of raters that will be dropped : 104378
The numbers of ratings that will be dropped : 622224
The numbers of remaining ratings : 527556


In [14]:
print(f"Total number of ratings : {len(df_ratings)}")
print(f"Total number of books : {len(df_books)}")
rated = df_ratings['isbn'].value_counts()
print(f"Total number of rated books : {len(rated)}")
rated.sort_values(ascending=False).head()
print(f"The numbers of rated books that will be dropped : {len(rated[rated < 100])}")
print(f"The numbers of books that will be dropped : {df_books['isbn'].isin(rated[rated < 100].index).sum()}")
df_ratings_rm = df_ratings_rm[~df_ratings_rm['isbn'].isin(rated[rated < 100].index)]
print(f"The numbers of remaining ratings : {len(df_ratings_rm)}")

Total number of ratings : 1149780
Total number of books : 271378
Total number of rated books : 340556
The numbers of rated books that will be dropped : 339825
The numbers of books that will be dropped : 269442
The numbers of remaining ratings : 49781


In [19]:
df = df_ratings_rm.pivot_table(index=['user'],columns=['isbn'],values='rating').fillna(0).T
df.index = df.join(df_books.set_index('isbn'))['title']
df = df.sort_index()
df.loc["The Queen of the Damned (Vampire Chronicles (Paperback))"][:5]

user
254     0.0
2276    0.0
2766    0.0
2977    0.0
3363    0.0
Name: The Queen of the Damned (Vampire Chronicles (Paperback)), dtype: float32

In [16]:
model = NearestNeighbors(metric='cosine')
model.fit(df.values)

NearestNeighbors(metric='cosine')

In [20]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
  try:
    book = df.loc[title]
  except KeyError as e:
    print(f"The book {e} doesn't exist!")
    return

  distance, indice = model.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : df.iloc[indice[0]].index.values,
      'distance': distance[0]
    }).sort_values(by='distance', ascending=False).head(5).values

  return [title, recommended_books]

In [18]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
